In [12]:
#tạo từ điển ưu tiên y,x
import xml.etree.ElementTree as ET
from collections import defaultdict

# Đường dẫn đến tệp XML
xml_file_path =  'D:\\Downloads\\scene\\SceneTrialTest\\words.xml'

# Đọc và phân tích tệp XML
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Khởi tạo từ điển lưu trữ thông tin theo imageName và các tag tương ứng
image_tag_dict = defaultdict(list)

# Duyệt qua mỗi phần tử <image>
for image_element in root.findall('.//image'):
    image_name = image_element.find('imageName').text
    
    # Khởi tạo danh sách lưu trữ thông tin vùng được gắn thẻ cho ảnh hiện tại
    tagged_info_list = []
    
    # Duyệt qua các phần tử <taggedRectangle> trong phần tử <image>
    for rectangle_element in image_element.findall('.//taggedRectangle'):
        x = float(rectangle_element.get('x'))
        y = float(rectangle_element.get('y'))
        
        x = round(x, -1)
        y = round(y, -1)
        
        # Trích xuất nội dung thẻ <tag>
        tag_text = rectangle_element.find('tag').text
        
        # Thêm thông tin vào danh sách
        tagged_info_list.append((y, x, tag_text))  # Sắp xếp theo (y, x) để ưu tiên đọc từ trên xuống, từ trái qua phải
    
    # Sắp xếp theo thứ tự của tọa độ và thêm vào từ điển
    sorted_info = sorted(tagged_info_list, key=lambda info: (info[0], info[1]))
    image_tag_dict[image_name] = [tag for _, _, tag in sorted_info]

for image_name, tags in image_tag_dict.items():
    print(f"Image: {image_name}")
    print(f"Tags: {tags}")
    print("-" * 20)


Image: ryoungt_05.08.2002/aPICT0034.JPG
Tags: ['ESSEX', 'SUMMER', 'SCHOOL', 'IN', 'DATA', 'ANALYSIS', '4', '311']
--------------------
Image: ryoungt_05.08.2002/aPICT0035.JPG
Tags: ['YOU', 'ARE', 'HERE']
--------------------
Image: ryoungt_05.08.2002/aPICT0036.JPG
Tags: ['DEPARTMENT', 'OF', 'GOVERNMENT', 'COURSES', 'STAFF', 'FOR']
--------------------
Image: ryoungt_05.08.2002/bPICT0007.JPG
Tags: ['Start', 'WinM', 'IIYAMA', 'POCKET', 'FLASH', 'READER']
--------------------
Image: ryoungt_05.08.2002/bPICT0008.JPG
Tags: ['Technology', 'for', 'life', 'jungle', 'com']
--------------------
Image: ryoungt_05.08.2002/bPICT0009.JPG
Tags: ['SNOUT']
--------------------
Image: ryoungt_05.08.2002/bPICT0010.JPG
Tags: ['graphic', '2', 'JAVA', 'JFC', 'Mastering', 'the', '3RD', 'EDITION']
--------------------
Image: ryoungt_05.08.2002/bPICT0012.JPG
Tags: ["WRIGLEY'S", 'DOUBLEMINT', 'CHEWING', 'GUM']
--------------------
Image: ryoungt_05.08.2002/bPICT0014.JPG
Tags: ['9', '12', 'September', '2002', 'O

In [13]:
#Duyệt thư mục ryoungt_05.08.2002
from os import listdir
from os.path import isfile, join
from paddleocr import PaddleOCR

#Hàm tính độ đánh giá
def jaccard_similarity(a, b):
    intersection = len(set(a) & set(b))
    union = len(set(a) | set(b))
    jaccard = intersection / union
    return jaccard

# Khởi tạo mô hình OCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Đường dẫn đến thư mục chứa ảnh
folder_path = 'D:\\Downloads\\scene\\SceneTrialTest\\ryoungt_05.08.2002'

# Lấy danh sách tệp trong thư mục
image_files = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]
count = 0 #đếm số ảnh trong thư mục
sum = 0 #lưu độ đánh giá

# Duyệt qua danh sách tệp và thực hiện nhận dạng
for image_file in image_files:
    img_path = join(folder_path, image_file)
    print(f"Đang nhận dạng ảnh: {img_path}")
    key = "ryoungt_05.08.2002/"+image_file;
    result = ocr.ocr(img_path, cls=False)
    print("Kết quả nhận dạng:")
    for idx in range(len(result)):
        res = result[idx]
        words = []
        for word_info in res:
            s = word_info[1][0]
            for i in s:
                if (not i.isalpha() and not i.isdigit() and i != " "):
                    s=s.replace(i," ")
            words = words+s.split() 
    print(words)
    print (image_tag_dict[key])
    
    reference_translation = image_tag_dict[key]
    candidate_translation = words
    # Chuyển tất cả các từ trong cả hai chuỗi về chữ thường
    reference_translation_lower = [word.lower() for word in reference_translation]
    candidate_translation_lower = [word.lower() for word in candidate_translation]
    sum = sum + jaccard_similarity(reference_translation, candidate_translation)
    print("Độ đo: ",jaccard_similarity(reference_translation, candidate_translation))
    count = count +1
    
    
    print("-" * 20)
    
print("Độ đo của ryoungt_05.08.2002:",sum/count)

[2023/08/21 23:47:39] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\Admin/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\Admin/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thre

Kết quả nhận dạng:
['EXCLUSIVELYFOR', 'WHEELCHAIR', 'USERS']
['EXCLUSIVELY', 'for', 'USERS', 'WHEELCHAIR']
Độ đo:  0.4
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_05.08.2002\bPICT0022.JPG
[2023/08/21 23:48:10] ppocr DEBUG: dt_boxes num : 1, elapse : 0.789107084274292
[2023/08/21 23:48:10] ppocr DEBUG: rec_res num  : 1, elapse : 0.2024085521697998
Kết quả nhận dạng:
['Metool']
['Metool']
Độ đo:  1.0
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_05.08.2002\bPict0027.jpg
[2023/08/21 23:48:10] ppocr DEBUG: dt_boxes num : 4, elapse : 0.6422905921936035
[2023/08/21 23:48:12] ppocr DEBUG: rec_res num  : 4, elapse : 1.3004248142242432
Kết quả nhận dạng:
['FIRE', 'BREAKGLASS', 'PRESSHERE', 'qpolo', 'xp95']
['FIRE', 'BREAK', 'GLASS', 'PRESS', 'HERE', 'apollo', 'XP', '95']
Độ đo:  0.08333333333333333
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_05.08.2002\Pict0001.jpg
[2023/08/21 23:4

[2023/08/21 23:48:42] ppocr DEBUG: dt_boxes num : 3, elapse : 1.1134142875671387
[2023/08/21 23:48:43] ppocr DEBUG: rec_res num  : 3, elapse : 0.9961984157562256
Kết quả nhận dạng:
['PRIVATE', 'CAR', 'PARK', 'LION', 'WALK', 'CHURCH']
['PRIVATE', 'PARK', 'CAR', 'CHURCH', 'WALK', 'LION']
Độ đo:  1.0
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_05.08.2002\PICT0024.JPG
[2023/08/21 23:48:44] ppocr DEBUG: dt_boxes num : 6, elapse : 0.8005375862121582
[2023/08/21 23:48:46] ppocr DEBUG: rec_res num  : 6, elapse : 1.7483024597167969
Kết quả nhận dạng:
['WHILE', 'YOU', 'WALT', 'SHOE', 'REPAIRS', 'AVAILABLE']
['WHILE', 'YOU', 'WAIT', 'SHOE', 'REPAIRS']
Độ đo:  0.5714285714285714
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_05.08.2002\PICT0025.JPG
[2023/08/21 23:48:46] ppocr DEBUG: dt_boxes num : 0, elapse : 0.7346851825714111
[2023/08/21 23:48:46] ppocr DEBUG: rec_res num  : 0, elapse : 0.0
Kết quả nhận dạng:
[]
['LITTER'

[2023/08/21 23:49:14] ppocr DEBUG: rec_res num  : 3, elapse : 0.5415873527526855
Kết quả nhận dạng:
['Coach', 'Stop', 'NATIONAL', 'EXPRESS']
['Coach', 'Stop', 'NATIONAL', 'N', 'EXPRESS']
Độ đo:  0.8
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_05.08.2002\PICTs0005.JPG
[2023/08/21 23:49:15] ppocr DEBUG: dt_boxes num : 1, elapse : 0.9749510288238525
[2023/08/21 23:49:15] ppocr DEBUG: rec_res num  : 1, elapse : 0.18250489234924316
Kết quả nhận dạng:
['First']
['f', 'First']
Độ đo:  0.5
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_05.08.2002\PICTs0006.JPG
[2023/08/21 23:49:16] ppocr DEBUG: dt_boxes num : 1, elapse : 0.7278051376342773
[2023/08/21 23:49:16] ppocr DEBUG: rec_res num  : 1, elapse : 0.1875016689300537
Kết quả nhận dạng:
['Used', 'tickets']
['Used', 'tickets']
Độ đo:  1.0
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_05.08.2002\PICTs0007.JPG
[2023/08/21 23:49:17] ppo

[2023/08/21 23:49:39] ppocr DEBUG: dt_boxes num : 2, elapse : 0.3500642776489258
[2023/08/21 23:49:40] ppocr DEBUG: rec_res num  : 2, elapse : 0.36410951614379883
Kết quả nhận dạng:
['P']
['P', 'S', '3']
Độ đo:  0.3333333333333333
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_05.08.2002\PICTs0032.JPG
[2023/08/21 23:49:40] ppocr DEBUG: dt_boxes num : 2, elapse : 0.36058664321899414
[2023/08/21 23:49:41] ppocr DEBUG: rec_res num  : 2, elapse : 0.7555079460144043
Kết quả nhận dạng:
['CounsellingHelpline', 'Lifelines', 'Family', 'Support']
['Counselling', 'helpline', 'Family', 'Support', 'Lifelines']
Độ đo:  0.5
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_05.08.2002\PICTs0033.JPG
[2023/08/21 23:49:41] ppocr DEBUG: dt_boxes num : 3, elapse : 0.4168837070465088
[2023/08/21 23:49:42] ppocr DEBUG: rec_res num  : 3, elapse : 0.9783751964569092
Kết quả nhận dạng:
['UK', 'DANCE', 'PROTOTYPE', 'INSPIRED', 'RECORDS']
['UK',

[2023/08/21 23:50:09] ppocr DEBUG: dt_boxes num : 4, elapse : 0.7564437389373779
[2023/08/21 23:50:09] ppocr DEBUG: rec_res num  : 4, elapse : 0.7269754409790039
Kết quả nhận dạng:
['CAFC', 'BAR', 'UCnUC', 'food']
['MONDO', 'BAR', 'VENUE', 'CAFE', 'food']
Độ đo:  0.2857142857142857
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_05.08.2002\wPICT0006.JPG
[2023/08/21 23:50:10] ppocr DEBUG: dt_boxes num : 4, elapse : 0.7699375152587891
[2023/08/21 23:50:11] ppocr DEBUG: rec_res num  : 4, elapse : 0.93597412109375
Kết quả nhận dạng:
['Sports', 'Centre', 'Wivennoe', 'ark', 'Conference', 'Centre', 'Car', 'Parks']
['Sports', 'Centre', 'Wivenhoe', 'Park', 'Conference', 'Centre', 'Car', 'Parks']
Độ đo:  0.5555555555555556
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_05.08.2002\wPICT0007.JPG
[2023/08/21 23:50:12] ppocr DEBUG: dt_boxes num : 4, elapse : 0.8207688331604004
[2023/08/21 23:50:13] ppocr DEBUG: rec_res num  : 4, 

In [17]:
#duyệt "ryoungt_13.08.2002"

#Hàm tính độ đánh giá
def jaccard_similarity(a, b):
    intersection = len(set(a) & set(b))
    union = len(set(a) | set(b))
    jaccard = intersection / union
    return jaccard

# Khởi tạo mô hình OCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Đường dẫn đến thư mục chứa ảnh
folder_path = 'D:\\Downloads\\scene\\SceneTrialTest\\ryoungt_13.08.2002'

# Lấy danh sách tệp trong thư mục
image_files = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]
count2 = 0 #đếm số ảnh trong thư mục
sum2 = 0 #lưu độ đánh giá

# Duyệt qua danh sách tệp và thực hiện nhận dạng
for image_file in image_files:
    img_path = join(folder_path, image_file)
    print(f"Đang nhận dạng ảnh: {img_path}")
    key = "ryoungt_13.08.2002/"+image_file;
    result = ocr.ocr(img_path, cls=False)
    print("Kết quả nhận dạng:")
    for idx in range(len(result)):
        res = result[idx]
        words = []
        for word_info in res:
            s = word_info[1][0]
            for i in s:
                if (not i.isalpha() and not i.isdigit() and i != " "):
                    s=s.replace(i," ")
            words = words+s.split() 
    print(words)
    print (image_tag_dict[key])
    
    reference_translation = image_tag_dict[key]
    candidate_translation = words
    # Chuyển tất cả các từ trong cả hai chuỗi về chữ thường
    reference_translation_lower = [word.lower() for word in reference_translation]
    candidate_translation_lower = [word.lower() for word in candidate_translation]
    sum2 = sum2 + jaccard_similarity(reference_translation, candidate_translation)
    print("Độ đo: ",jaccard_similarity(reference_translation, candidate_translation))
    count2 = count2 +1
    
    
    print("-" * 20)

print("Độ đo của ryoungt_13.08.2002:",sum2/count2)

[2023/08/21 23:52:42] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\Admin/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\Admin/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thre

[2023/08/21 23:52:59] ppocr DEBUG: dt_boxes num : 1, elapse : 0.37795495986938477
[2023/08/21 23:52:59] ppocr DEBUG: rec_res num  : 1, elapse : 0.3920261859893799
Kết quả nhận dạng:
['APPLICATION', 'FORM']
['APPLICATION', 'FORM.']
Độ đo:  0.3333333333333333
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_13.08.2002\dpCT0027.jpg
[2023/08/21 23:53:00] ppocr DEBUG: dt_boxes num : 4, elapse : 0.8505344390869141
[2023/08/21 23:53:02] ppocr DEBUG: rec_res num  : 4, elapse : 1.529571294784546
Kết quả nhận dạng:
['FIRE', 'BREAK', 'GLASS', 'PRESSHERE', 'qpolo', 'xp95']
['FIRE', 'GLASS', 'BREAK', 'PRESS', 'HERE', 'apollo', 'XP', '95']
Độ đo:  0.2727272727272727
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_13.08.2002\dpCT0030.jpg
[2023/08/21 23:53:03] ppocr DEBUG: dt_boxes num : 1, elapse : 0.7290868759155273
[2023/08/21 23:53:03] ppocr DEBUG: rec_res num  : 1, elapse : 0.29720568656921387
Kết quả nhận dạng:
['BLACK', 'PEPPE

[2023/08/21 23:53:25] ppocr DEBUG: dt_boxes num : 4, elapse : 0.36698341369628906
[2023/08/21 23:53:26] ppocr DEBUG: rec_res num  : 4, elapse : 1.3980011940002441
Kết quả nhận dạng:
['MTHE', 'ESSENCE', '14J0NE2002', 'THE', 'UK', 'S', 'No', '1', 'HARD', 'HOUSE', 'DJ', 'IS', 'BACK']
['KARTOUCHE', 'ESSENCE', '14', 'JUNE', '2002', 'THE', "UK'S", 'No.1', 'HARD', 'HOUSE', 'DJ', 'IS', 'BACK!']
Độ đo:  0.3
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_13.08.2002\dPICT0038.JPG
[2023/08/21 23:53:27] ppocr DEBUG: dt_boxes num : 1, elapse : 0.3549685478210449
[2023/08/21 23:53:27] ppocr DEBUG: rec_res num  : 1, elapse : 0.20707249641418457
Kết quả nhận dạng:
['30', '6']
['30', '6"']
Độ đo:  0.3333333333333333
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_13.08.2002\dPICT0039.JPG
[2023/08/21 23:53:27] ppocr DEBUG: dt_boxes num : 1, elapse : 0.37052440643310547
[2023/08/21 23:53:28] ppocr DEBUG: rec_res num  : 1, elapse : 0.46

[2023/08/21 23:53:47] ppocr DEBUG: dt_boxes num : 4, elapse : 0.3196678161621094
[2023/08/21 23:53:49] ppocr DEBUG: rec_res num  : 4, elapse : 2.13509464263916
Kết quả nhận dạng:
['SLUSH', 'PUPPiE', 'ASCOOLASIT', 'GETS']
['SLUSH', 'PUPPiE', 'AS', 'COOL', 'AS', 'IT', 'GETS']
Độ đo:  0.42857142857142855
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_13.08.2002\dPICT0064.JPG
[2023/08/21 23:53:50] ppocr DEBUG: dt_boxes num : 2, elapse : 0.3385043144226074
[2023/08/21 23:53:50] ppocr DEBUG: rec_res num  : 2, elapse : 0.4677762985229492
Kết quả nhận dạng:
['PIZZA', 'GALORE', 'Cheques']
['PIZZA', 'GALORE', 'Cheques']
Độ đo:  1.0
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_13.08.2002\dPICT0065.JPG
[2023/08/21 23:53:51] ppocr DEBUG: dt_boxes num : 1, elapse : 0.32114720344543457
[2023/08/21 23:53:51] ppocr DEBUG: rec_res num  : 1, elapse : 0.25232982635498047
Kết quả nhận dạng:
['The', 'Adult', 'Community', 'College']
['

Kết quả nhận dạng:
['POST', 'OFFIC', 'NATIONALEXPRESS']
['POST', 'OFFICE', 'NATIONAL', 'EXPRESS']
Độ đo:  0.16666666666666666
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_13.08.2002\fdPICT0011.JPG
[2023/08/21 23:54:13] ppocr DEBUG: dt_boxes num : 4, elapse : 0.705608606338501
[2023/08/21 23:54:14] ppocr DEBUG: rec_res num  : 4, elapse : 0.8178915977478027
Kết quả nhận dạng:
['POSTPAK', 'POSTPAK', 'POST', 'OFFICE']
['POSTPAK', 'POSTPAK', 'POST', 'OFFICE']
Độ đo:  1.0
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_13.08.2002\fdPICT0014.JPG
[2023/08/21 23:54:15] ppocr DEBUG: dt_boxes num : 2, elapse : 0.9737818241119385
[2023/08/21 23:54:15] ppocr DEBUG: rec_res num  : 2, elapse : 0.3615455627441406
Kết quả nhận dạng:
['copy', 'centre', 'the']
['centre', 'copy', 'the']
Độ đo:  1.0
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_13.08.2002\fdPICT0015.JPG
[2023/08/21 23:54:16] ppocr 

[2023/08/21 23:54:34] ppocr DEBUG: rec_res num  : 1, elapse : 0.18849492073059082
Kết quả nhận dạng:
['CELMAC']
['ILET', 'CELMAC', 'AT']
Độ đo:  0.3333333333333333
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_13.08.2002\rPICT0001.JPG
[2023/08/21 23:54:34] ppocr DEBUG: dt_boxes num : 2, elapse : 0.7120981216430664
[2023/08/21 23:54:35] ppocr DEBUG: rec_res num  : 2, elapse : 0.3627464771270752
Kết quả nhận dạng:
['HSBCX', 'tiqroupy']
['HSBC', 'citigroup']
Độ đo:  0.0
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_13.08.2002\rPICT0004.JPG
[2023/08/21 23:54:35] ppocr DEBUG: dt_boxes num : 2, elapse : 0.7122738361358643
[2023/08/21 23:54:36] ppocr DEBUG: rec_res num  : 2, elapse : 0.6144511699676514
Kết quả nhận dạng:
['THE', 'SOUTH', 'COLONNADE', 'E14']
['THE', 'SOUTH', 'COLONNADE', 'E14']
Độ đo:  1.0
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_13.08.2002\rPICT0005.JPG
[2023/08

[2023/08/21 23:55:05] ppocr DEBUG: rec_res num  : 4, elapse : 1.9538066387176514
Kết quả nhận dạng:
['9', '12', 'September', '2002', 'Oldenburg', 'Gern', 'FTRTFT', '2002', '74']
['12.', 'September', '9.', '2002,', 'Oldenburg,', 'Germ', 'FTRTFT', '2002']
Độ đo:  0.23076923076923078
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_13.08.2002\vPICT0015.JPG
[2023/08/21 23:55:06] ppocr DEBUG: dt_boxes num : 4, elapse : 0.7187905311584473
[2023/08/21 23:55:07] ppocr DEBUG: rec_res num  : 4, elapse : 1.3134853839874268
Kết quả nhận dạng:
['MAXIMUM', 'LOAD', '16OO', 'KG', 'OR', '22', 'PERSONS', 'GUiueLinue']
['LOAD', '1600', 'KG', 'MAXIMUM', 'OR', '22', 'PERSONS', 'GUIDELINE']
Độ đo:  0.6
--------------------
Đang nhận dạng ảnh: D:\Downloads\scene\SceneTrialTest\ryoungt_13.08.2002\vPICT0016.JPG
[2023/08/21 23:55:08] ppocr DEBUG: dt_boxes num : 1, elapse : 0.705120325088501
[2023/08/21 23:55:08] ppocr DEBUG: rec_res num  : 1, elapse : 0.1845417022705078
Kết quả

In [18]:
#Duyệt thư mục "sml_01.08.2002"

#Hàm tính độ đánh giá
def jaccard_similarity(a, b):
    intersection = len(set(a) & set(b))
    union = len(set(a) | set(b))
    jaccard = intersection / union
    return jaccard

# Khởi tạo mô hình OCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Đường dẫn đến thư mục chứa ảnh
folder_path = 'D:\\Downloads\\scene\\SceneTrialTest\\sml_01.08.2002'

# Lấy danh sách tệp trong thư mục
image_files = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]
count3 = 0 #đếm số ảnh trong thư mục
sum3 = 0 #lưu độ đánh giá

# Duyệt qua danh sách tệp và thực hiện nhận dạng
for image_file in image_files:
    img_path = join(folder_path, image_file)
    print(f"Đang nhận dạng ảnh: {img_path}")
    key = "sml_01.08.2002/"+image_file;
    result = ocr.ocr(img_path, cls=False)
    print("Kết quả nhận dạng:")
    for idx in range(len(result)):
        res = result[idx]
        words = []
        for word_info in res:
            s = word_info[1][0]
            for i in s:
                if (not i.isalpha() and not i.isdigit() and i != " "):
                    s=s.replace(i," ")
            words = words+s.split() 
    print(words)
    print (image_tag_dict[key])
    
    reference_translation = image_tag_dict[key]
    candidate_translation = words
    # Chuyển tất cả các từ trong cả hai chuỗi về chữ thường
    reference_translation_lower = [word.lower() for word in reference_translation]
    candidate_translation_lower = [word.lower() for word in candidate_translation]
    sum3 = sum3 + jaccard_similarity(reference_translation, candidate_translation)
    print("Độ đo:", jaccard_similarity(reference_translation, candidate_translation))
    count3 = count3 +1
    
    
    print("-" * 20)

print("Độ đo của sml_01.08.2002:",sum3/count3)


[2023/08/21 23:56:06] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\Admin/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\Admin/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thre

In [19]:
print("Độ đo đánh giá tập dataset đánh giá là: ", (((sum/count)+(sum2/count2)+(sum3/count3))/3))

Độ đo đánh giá tập dataset đánh giá là:  0.6789651369721154
